# Estimate floor area projections based on GVA and dwellings/households

First cut estimate for energy demand model input.

Requires inputs:
- dwellings scenarios (see `arc-dwellings`)
- gva scenarios (see `arc-economics`)

In [ ]:
import glob
import os

import pandas

In [ ]:
AVERAGE_DWELLING_M2 = 85  # based on average home size
AVERAGE_NONRES_M2_PER_MGBP_GVA = 1  # based on no data or estimate - to be used in elasticity-based model

In [ ]:
data_path = os.path.join('.', 'data_as_provided')
output_path = os.path.join('.', 'data_processed')

In [ ]:
dwellings_filenames = glob.glob(os.path.join(data_path, 'arc_dwellings__*.csv'))
econ_filenames = glob.glob(os.path.join(data_path, 'arc_gva_employment__*.csv'))

In [ ]:
def estimate_floor_area(dwellings, gva):
    floor_area = dwellings.merge(gva, on=['timestep', 'lad_uk_2016'])
    floor_area['residential'] = floor_area.dwellings * AVERAGE_DWELLING_M2
    floor_area['non_residential'] = floor_area.gva * AVERAGE_NONRES_M2_PER_MGBP_GVA
    floor_area = floor_area[
        ['timestep', 'lad_uk_2016', 'residential' ,'non_residential']
    ].melt(
        id_vars=['timestep', 'lad_uk_2016'],
        var_name='residential_or_non',
        value_name='floor_area'
    )
    
    return floor_area

In [ ]:
for d, e in zip(dwellings_filenames, econ_filenames):
    d_df = pandas.read_csv(d)
    print(d_df.head(), len(d_df))
    e_df = pandas.read_csv(e)
    print(e_df.head(), len(e_df))
    key = os.path.basename(d).replace('arc_dwellings__', '').replace('.csv', '')
    key2 = os.path.basename(e).replace('arc_gva_employment__', '').replace('.csv', '')
    assert key == key2
    f_df = estimate_floor_area(d_df, e_df)
    print(len(f_df), len(f_df.timestep.unique()), len(f_df.lad_uk_2016.unique()))
    f_df.to_csv(os.path.join(output_path, 'arc_floor_area__{}.csv'.format(key)))
    break